In [ ]:
from okx import PublicData

public = PublicData.PublicAPI(flag='0')
data = public.get_instruments('SWAP')['data']
names = [x['instId'] for x in data if  "USDT-SWAP" in x['instId']]

['BTC-USDT-SWAP', 'ETH-USDT-SWAP', 'OKB-USDT-SWAP', 'SOL-USDT-SWAP', 'DOGE-USDT-SWAP', 'XRP-USDT-SWAP', 'BCH-USDT-SWAP', 'PUMP-USDT-SWAP', 'WLFI-USDT-SWAP', 'LINEA-USDT-SWAP', '0G-USDT-SWAP', '1INCH-USDT-SWAP', '2Z-USDT-SWAP', 'A-USDT-SWAP', 'AAVE-USDT-SWAP', 'ACE-USDT-SWAP', 'ACH-USDT-SWAP', 'ACT-USDT-SWAP', 'ADA-USDT-SWAP', 'AERO-USDT-SWAP', 'AEVO-USDT-SWAP', 'AGLD-USDT-SWAP', 'AIXBT-USDT-SWAP', 'ALGO-USDT-SWAP', 'ALLO-USDT-SWAP', 'ANIME-USDT-SWAP', 'APE-USDT-SWAP', 'API3-USDT-SWAP', 'APR-USDT-SWAP', 'APT-USDT-SWAP', 'AR-USDT-SWAP', 'ARB-USDT-SWAP', 'ARKM-USDT-SWAP', 'ASTER-USDT-SWAP', 'AT-USDT-SWAP', 'ATH-USDT-SWAP', 'ATOM-USDT-SWAP', 'AUCTION-USDT-SWAP', 'AVAX-USDT-SWAP', 'AVNT-USDT-SWAP', 'AXS-USDT-SWAP', 'BABY-USDT-SWAP', 'BAND-USDT-SWAP', 'BARD-USDT-SWAP', 'BAT-USDT-SWAP', 'BEAT-USDT-SWAP', 'BERA-USDT-SWAP', 'BICO-USDT-SWAP', 'BIGTIME-USDT-SWAP', 'BIO-USDT-SWAP', 'BLUR-USDT-SWAP', 'BLUAI-USDT-SWAP', 'BNB-USDT-SWAP', 'BOME-USDT-SWAP', 'BONK-USDT-SWAP', 'BRETT-USDT-SWAP', 'CAT-USD

In [13]:
# from okx import TradingData

# api = TradingData.TradingDataAPI(flag='0')
# api.get_margin_lending_ratio(ccy="ETH")

In [ ]:
import okx.MarketData as MarketData
import pandas as pd

market = MarketData.MarketAPI(flag='0')
def get_candles(instId='ETH-USDT-SWAP', bar='1H', limit="100"):
    result = market.get_candlesticks(instId=instId, bar=bar, limit=limit)

    data = result['data']  # 列表，每项：[ts, o, h, l, c, vol, volCcy, ...]
    df = pd.DataFrame(data, columns=[
        'ts', 'open', 'high', 'low', 'close', 
        'vol', 'volCcy', 'volCcyQuote', 'confirm'
    ])
    df = df.loc[:, ["open", "high", "low", "close"]]
    df = df.astype({'open':float, 'high':float, 'low':float, 'close':float})
    df = df[::-1].reset_index(drop=True)    # 最后一行为新k

    return df

,open,high,low,close
0,3029.66,3032.85,2992.00,3010.05
1,3010.06,3019.80,2992.56,3014.54
2,3014.55,3026.80,3000.72,3022.50
3,3022.50,3031.38,3009.28,3027.23
4,3027.23,3049.59,2994.17,2998.66
...,...,...,...,...
95,2828.84,2841.84,2820.56,2837.52
96,2837.51,2837.52,2786.09,2796.79
97,2796.79,2806.14,2792.35,2798.17
98,2798.16,2807.43,2788.51,2801.92


In [135]:
import pandas as pd
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')

def tr(data):
    data['previous_close'] = data['close'].shift(1)
    data['high-low'] = abs(data['high'] - data['low'])
    data['high-pc'] = abs(data['high'] - data['previous_close'])
    data['low-pc'] = abs(data['low'] - data['previous_close'])

    tr = data[['high-low', 'high-pc', 'low-pc']].max(axis=1)

    return tr

def atr(data, period):
    data['tr'] = tr(data)
    atr = data['tr'].rolling(period).mean()

    return atr

def supertrend(df, period=10, atr_multiplier=3):
    hl2 = (df['high'] + df['low']) / 2
    df['atr'] = atr(df, period)
    df['upperband'] = hl2 + (atr_multiplier * df['atr'])
    df['lowerband'] = hl2 - (atr_multiplier * df['atr'])
    df['in_uptrend'] = True

    for current in range(1, len(df.index)):
        previous = current - 1

        if df['close'][current] > df['upperband'][previous]:
            df['in_uptrend'][current] = True
        elif df['close'][current] < df['lowerband'][previous]:
            df['in_uptrend'][current] = False
        else:
            df['in_uptrend'][current] = df['in_uptrend'][previous]

            if df['in_uptrend'][current] and df['lowerband'][current] < df['lowerband'][previous]:
                df['lowerband'][current] = df['lowerband'][previous]

            if not df['in_uptrend'][current] and df['upperband'][current] > df['upperband'][previous]:
                df['upperband'][current] = df['upperband'][previous]
        
    return df

In [136]:
supertrend(df).tail(20).iloc[:, -1]

180    False
181    False
182    False
183    False
184    False
185    False
186    False
187    False
188    False
189    False
190    False
191    False
192    False
193    False
194    False
195    False
196    False
197    False
198    False
199    False
Name: in_uptrend, dtype: bool